In [309]:
!pip install qiskit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [310]:
from qiskit import QuantumRegister, ClassicalRegister
from qiskit import QuantumCircuit, execute,IBMQ
from qiskit.tools.monitor import job_monitor
from qiskit.circuit.library import QFT
from qiskit.circuit.library.standard_gates import PhaseGate
from qiskit import Aer
import numpy as np

backend = Aer.get_backend('statevector_simulator')
   

In [311]:
def inial(qr,q,qc):
  i = len(q)
  for j in q:
    if j=='1':
     qc.x(qr[i-1])
    i=i-1

In [320]:
def multiplier(q1,q2):
  q1="{0:b}".format(int(q1))
  q2="{0:b}".format(int(q2))
  print(len(q1))
  num_state_qubits=max(len(q1),len(q2))
  num_result_qubits=5
  from qiskit.circuit import QuantumRegister, QuantumCircuit
  num_qubits=num_state_qubits
  qr_a = QuantumRegister(num_state_qubits, name="a")
  qr_b = QuantumRegister(num_state_qubits, name="b")
  qr_out = QuantumRegister(num_result_qubits, name="out")
  c = ClassicalRegister(num_result_qubits,'c')
  circuit=QuantumCircuit()
  circuit.add_register(qr_a, qr_b, qr_out,c)
  inial(qr_a,q1,circuit)
  inial(qr_b,q2,circuit)
  # build multiplication circuit

  circuit.append(QFT(num_result_qubits, do_swaps=False).to_gate(), qr_out[:])


  for j in range(1, num_state_qubits + 1):
        for i in range(1, num_state_qubits + 1):
              for k in range(1, num_result_qubits + 1):
                   lam = (2 * np.pi) / (2 ** (i + j + k - 2 * num_state_qubits))
                   circuit.append(
                   PhaseGate(lam).control(2),
                   [qr_a[num_state_qubits - j], qr_b[num_state_qubits - i], qr_out[k - 1]],
                             )

  circuit.append(QFT(num_result_qubits, do_swaps=False).inverse().to_gate(), qr_out[:])
  for i in range(num_result_qubits):
      circuit.measure(qr_out[int(i)],c[i])
 

  print(circuit)

  job = execute(circuit, backend, shots=2000)
  result = job.result()
  counts = result.get_counts()
  ans=list(counts.keys())
  z=0
  a=0
  for i in "".join(reversed(ans[0])):
    z+=int(i)*(2**a)
    a+=1
  return z

In [321]:
multiplier(10,2)

4
                                                                            »
  a_0: ─────────────────────────────────────────────────────────────────────»
        ┌───┐                                                               »
  a_1: ─┤ X ├───────────────────────────────────────────────────────────────»
        └───┘                                                               »
  a_2: ─────────────────────────────────────────────────────────────────────»
        ┌───┐                                                               »
  a_3: ─┤ X ├───■────────■────────■────────■───────■───────■────────■───────»
        └───┘   │        │        │        │       │       │        │       »
  b_0: ─────────┼────────┼────────┼────────┼───────┼───────┼────────┼───────»
        ┌───┐   │        │        │        │       │       │        │       »
  b_1: ─┤ X ├───┼────────┼────────┼────────┼───────┼───────┼────────┼───────»
        └───┘   │        │        │        │       │       │  

20